In [ ]:
#@title 查看分配到的设备
# 查看设备，是K80还是T4,如果是K80...！
! /opt/bin/nvidia-smi

y
Sat Apr  3 13:33:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----

# 第一步 准备好workspace
这一步你可以选着两种方式
1. 使用默认的workspace，你无需自己上传，仅用于熟悉操作。
2. 通过Google Drive （谷歌云盘）上传自己的workspace到指定目录。
  

In [ ]:
#@title 开始挂载 
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#创建DeepFaceLab目录，并且进入目录
%cd /content/drive/My Drive/
!mkdir DeepFaceLab
%cd /content/drive/My Drive/DeepFaceLab/


Mounted at /content/drive
/content/drive/My Drive
mkdir: cannot create directory ‘DeepFaceLab’: File exists
/content/drive/My Drive/DeepFaceLab


这一环节有两个选择：  
如果你第一次使用，可以加载示例项目，然后往后运行。  
如果你是二次运行，就不需要加载了。  
如果你希望上传本地素材，可以先用pack脚本把素材打包成faceset.pak到aligned目录，无须解压，可直接进行训练和转换，加载速度会很快。


In [ ]:
#@title 加载示例项目workspace 
# 1. 练手可以使用这一行，直接git clone一个workspace
%cd /content/drive/My Drive/DeepFaceLab/
!rm -rf workspace
!git clone https://github.com/dream80/DFLWorkspace.git workspace

/content/drive/My Drive/DeepFaceLab
Cloning into 'workspace'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 39 (delta 0), reused 2 (delta 0), pack-reused 35
Unpacking objects: 100% (39/39), done.
Checking out files: 100% (8/8), done.


#第二步 安装DeepFaceLab
获取源代码，安装依赖,根据自己的情况选择版本  

last:最新版  

In [ ]:
#@title 开始安装

Version = "last" #@param ["v2.02.03", "v2.02.23b", "v2.02.28","v2.03.07","v2.03.15","v2.06.01","v2.06.19","last"]
%cd /content/drive/My Drive/DeepFaceLab
!rm -fr DeepFaceLab_Colab
!pip uninstall -y tensorflow
if Version=="620":
  print("620版加载中....")
  # 获取DFL源代码v1.6.1稳定版
  !git clone -b v1.6.1 https://github.com/dream80/DeepFaceLab_Colab.git
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  !pip install -r requirements-colab.txt  
elif Version=="last":
  print("最新版加载中....")
  !git clone https://github.com/iperov/DeepFaceLab.git  DeepFaceLab_Colab
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  !pip install -r requirements-colab.txt
else:
  print(Version+"加载中....")
  cmd="clone -b "+Version+" https://github.com/dream80/DeepFaceLab_Colab.git"  
  !git $cmd
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  !pip install -r requirements-colab.txt

!pip install --upgrade scikit-image
#!sudo apt-get install cuda-10-0


# 第三步. 提取脸部

这个步骤其实是分两次的。第一次选src，第二次选dst，很多对Deepfacelab不了解的人可能只点了一个，后面就会报错了。  
  
这一步，其实有三个小步骤，分解视频，提取人脸，打包素材。 如果有需要可以将他们拆分开来。一般是不用这么做的。 





In [ ]:
#@title 开始提取
target = "dst" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/
  !python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd
  !python main.py util --input-dir ../workspace/data_src/aligned  --pack-faceset
else:
  !python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/
  !python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --output-debug
  !python main.py util --input-dir ../workspace/data_dst/aligned  --pack-faceset


/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
[0] Enter FPS ( ?:help ) : 
0
[png] Output image format ( png/jpg ?:help ) : 
png
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame

In [ ]:
#@title 开始排序
target = "dst" #@param ["src","dst"]

if target=="src":
  #Src排序，可以通过谷歌云盘查看结果，删除不良图片
  cmd = "main.py sort --input-dir ../workspace/data_src/aligned"
  
else:
  cmd = "main.py sort --input-dir ../workspace/data_dst/aligned "

!python $cmd

Running sort tool.

Choose sorting method:
[0] blur
[1] motion_blur
[2] face yaw direction
[3] face pitch direction
[4] face rect size in source image
[5] histogram similarity
[6] histogram dissimilarity
[7] brightness
[8] hue
[9] amount of black pixels
[10] original filename
[11] one face in image
[12] absolute pixel difference
[13] best faces
[14] best faces faster

[5] : 
5
Sorting by histogram similarity...
Running on 1 threads


Sorting: 1it [00:00, 30.04it/s]   
Done.


# 第四步. 训练模型


*  支持SAEHD,Quick96等模型，根据自己的情况选择模型。
*  第一次需要配置模型参数，不懂的直接回车默认。
*  不想训练了可以点击停止，停止时会抛出异常，但是没什么关系。下次可以继续训练

In [ ]:
#@title 开始训练
Model = "SAEHD" #@param ["SAEHD","Quick96"]

%cd "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab/"
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd

/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
Running trainer.

[new] No saved models found. Enter a name of a new model : wo
wo

Model first run.

Choose one or several GPU idxs (separated by comma).

[CPU] : CPU
  [0] : Tesla P100-PCIE-16GB

[0] Which GPU indexes to choose? : 0
0

[0] Autobackup every N hour ( 0..24 ?:help ) : 
0
[n] Write preview history ( y/n ?:help ) : 
n
[0] Target iteration : 
0
[n] Flip SRC faces randomly ( y/n ?:help ) : 
n
[y] Flip DST faces randomly ( y/n ?:help ) : 
y
[8] Batch_size ( ?:help ) : 
8
[128] Resolution ( 64-640 ?:help ) : 
128
[f] Face type ( h/mf/f/wf/head ?:help ) : 
f
[liae-ud] AE architecture ( ?:help ) : 
liae-ud
[256] AutoEncoder dimensions ( 32-1024 ?:help ) : 
256
[64] Encoder dimensions ( 16-256 ?:help ) : 
64
[64] Decoder dimensions ( 16-256 ?:help ) : 
64
[22] Decoder mask dimensions ( 16-256 ?:help ) : 
22
[n] Eyes and mouth priority ( y/n ?:help ) : 
n
[n] Uniform yaw distribution of samples ( y/n ?:help ) : 
n
[y] Place mo

# 第五步. 转换输出

训练模型的时候选了什么，这里就选什么  
比如训练H128,这里就选H128


In [2]:
#@title 开始转换图片

Model = "SAEHD" #@param ["SAEHD", "Quick96"]
%cd "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab/"
cmd = " main.py merge --output-mask-dir ../workspace/data_dst/merged_mask --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model
!python $cmd

/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
Running merger.

Choose one of saved models, or enter a name to create a new model.
[r] : rename
[d] : delete

[0] : wo - latest
 : 0
0
Loading wo_SAEHD model...

Choose one or several GPU idxs (separated by comma).

[CPU] : CPU
  [0] : Tesla T4

[0] Which GPU indexes to choose? : 0
0

Initializing models: 100% 4/4 [00:08<00:00,  2.01s/it]
=========== Model Summary ===========
==                                 ==
==            Model name: wo_SAEHD ==
==                                 ==
==     Current iteration: 32939    ==
==                                 ==
==--------- Model Options ---------==
==                                 ==
==            resolution: 128      ==
==             face_type: f        ==
==     models_opt_on_gpu: True     ==
==                 archi: liae-ud  ==
==               ae_dims: 256      ==
==                e_dims: 64       ==
==                d_dims: 64       ==
==           d_mask_dims: 22      

In [3]:
#@title 开始图片转视频
!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4 --include-audio
!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged_mask --output-file ../workspace/result_mask.mp4 --reference-file ../workspace/data_dst.mp4 --include-audio --lossless

[16] Bitrate of output file in MB/s : 
16
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --en

#第六步. 继续训练
当你第二次开始训练，或者掉线之后继续训练时并不需要执行上面所有的步骤。只需要下面简单的几个步骤。
1. 挂载云盘
2. 安装依赖
3. 开始训练  


In [ ]:
#@title 一键运行
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

Model = "SAEHD" #@param ["SAEHD" , "Quick96"]


from google.colab import drive
drive.mount('/content/drive', force_remount=True)


# 进入DeepFaceLab_Colab目录
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

# 安装Python依赖
!pip uninstall -y tensorflow
!pip install -r requirements-colab.txt
!pip install --upgrade scikit-image
#!sudo apt-get install cuda-10-0

# 开始训练SAE ，如果是其他模型，修改后面的参数即可，比如H128。
#cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
#!python $cmd 

# 工具



打包素材。可以加快上传，下载，加载的速度。

In [ ]:
#@title 素材打包

target = "src" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned  --pack-faceset
else:
  !python main.py util --input-dir ../workspace/data_dst/aligned  --pack-faceset
 
  

/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
Performing faceset packing...

/content/drive/My Drive/DeepFaceLab/workspace/data_src/aligned/faceset.pak : file already exists !
Press enter to continue and overwrite.

Loading samples: 0it [00:00, ?it/s]
Processing: 0it [00:00, ?it/s]
Packing: 0it [00:00, ?it/s]
Deleting files: 0it [00:00, ?it/s]
Done.


In [ ]:
#@title 素材解包

target = "src" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned --unpack-faceset
else:
  !python main.py util --input-dir ../workspace/data_dst/aligned --unpack-faceset
 
  

/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
Performing faceset unpacking...

Unpacking: 0it [00:00, ?it/s]
Done.


In [ ]:
#@title 素材增强

target = "src" #@param ["src"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py facesettool enhance --input-dir ../workspace/data_src/aligned
else:
  !python main.py facesettool enhance --input-dir ../workspace/data_dst/aligned
 
  

/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

Choose one or several GPU idxs (separated by comma).

[CPU] : CPU
  [0] : Tesla P100-PCIE-16GB

[0] Which GPU indexes to choose? : 0
0

Enhancing faceset in data_src/aligned
Processing to data_src/aligned_enhanced
Running on Tesla P100-PCIE-16GB.
0it [00:00, ?it/s]
[y] 
Merge data_src/aligned_enhanced to data_src/aligned ? ( y/n ) : y
Copying processed files to data_src/aligned
Removing data_src/aligned_enhanced
Done.
